In [ ]:
# (c) Copyright M. Rizky Luthfianto. 2016

from __future__ import division

try:
    import pickle
except:
    import cPickle as pickle

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from keras.activations import hard_sigmoid
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, History

onehot = np.arange(0,21)
PSSM = np.arange(35,56)
features = np.hstack((onehot, PSSM))
labels = np.arange(22,30)

cull_ = np.load('cullpdb+profile_6133.npy')
cull = np.reshape(cull_, (-1, 700, 57))

X_train    = cull[0:5600, :, features]
y_train    = cull[0:5600, :, labels]
train_mask = cull[0:5600, :, 30] * -1 + 1

X_test    = cull[5605:5887, :, features]
y_test    = cull[5605:5887, :, labels]
test_mask = cull[5605:5887, :, 30] * -1 + 1

X_val    = cull[5877:6133, :, features]
y_val    = cull[5877:6133, :, labels]
val_mask = cull[5877:6133, :, 30] * -1 + 1

def masked_accuracy(y_true, y_pred, data_mask):
    sames = np.equal(np.argmax(y_true, axis=-1), np.argmax(y_pred, axis=-1)) # is Array
    filtered = sames * data_mask
    return np.sum(filtered)/np.sum(data_mask)

def masked_val_accuracy(y_true, y_pred):
    return masked_accuracy(y_true, y_pred, val_mask)

def masked_cb_accuracy(y_true, y_pred):
    return masked_accuracy(y_true, y_pred, cb_mask)

n_inputs = 42
num_classes = 8
seq_len = 700
from keras.regularizers import l2

#l2_num=1
#l2_num=0.1
#l2_num=0.01
#l2_num=0.001
l2_num=0

if l2_num==0:
    l2l=None
else:
    l2l=l2(l2_num)

from tensorflow_ops import MyAtrousConvolution1D, make_parallel
from keras.layers import Activation, Convolution1D, Dense, Dropout, Input, merge
from keras.layers import AtrousConvolution1D
from keras.models import Model

num_protein_features = 42
num_filters = 100
num_label=8

init='he_normal'

def WavenetBlock(n_atrous_filters, atrous_filter_size, atrous_rate):
    def f(input_):
        residual = input_
        tanh_out    = MyAtrousConvolution1D(n_atrous_filters, atrous_filter_size, init=init,
                                            atrous_rate=atrous_rate,
                                            border_mode='same',# dim_ordering='tf',
                                            activation='tanh')(input_)
        sigmoid_out = MyAtrousConvolution1D(n_atrous_filters, atrous_filter_size, init=init,
                                            atrous_rate=atrous_rate,
                                            border_mode='same',#  dim_ordering='tf',
                                            activation='sigmoid')(input_)
        merged = merge([tanh_out, sigmoid_out], mode='mul')

        skip_out = Convolution1D(num_protein_features, 1, activation='relu', border_mode='same', init=init)(merged)
        out = merge([skip_out, residual], mode='sum')
        return out, skip_out
    return f

def GluBlock(n_atrous_filters, atrous_filter_size, atrous_rate):
    def f(input_):
        residual = input_
        linear_out  = MyAtrousConvolution1D(n_atrous_filters, atrous_filter_size, init=init,
                                            atrous_rate=atrous_rate,
                                            border_mode='same',# dim_ordering='tf',
                                            activation='linear')(input_)
        sigmoid_out = MyAtrousConvolution1D(n_atrous_filters, atrous_filter_size, init=init,
                                            atrous_rate=atrous_rate,
                                            border_mode='same',#  dim_ordering='tf',
                                            activation='sigmoid')(input_)
        merged = merge([linear_out, sigmoid_out], mode='mul')

        skip_out = Convolution1D(num_protein_features, 1, activation='relu', border_mode='same', init=init)(merged)
        out = merge([skip_out, residual], mode='sum')
        return out, skip_out
    return f

num_timesteps = 700

def create_model(gate='wave'):
    input_ = Input(shape=(num_timesteps, num_protein_features))

    if gate=='glu':
        A, B = GluBlock(num_filters, atrous_filter_size=2, atrous_rate=2)(input_)        
    elif gate=='wave':
        A, B = WavenetBlock(num_filters, atrous_filter_size=2, atrous_rate=2)(input_)
    
    skip_connections = [B]
    for i in range(30):
        atrous_rate = 2**(i%10)
        if gate=='glu':
            A, B = GluBlock(num_filters, 2, atrous_rate)(A)
        elif gate=='wave':
            A, B = WavenetBlock(num_filters, 2, atrous_rate)(A)
        skip_connections.append(B)

    net = merge(skip_connections, mode='sum')
    net = Activation('relu')(net)

    net = Convolution1D(num_label, 1, activation='relu', init=init)(net)
    net = Convolution1D(num_label, 1, init=init)(net)
    #net = Dropout(0.5)(net)
    net = Dense(num_label, activation='softmax', init=init)(net)
    model = Model(input=input_, output=net)
    return model

model = create_model('wave')

from keras.optimizers import RMSprop, Adam
optimizer = RMSprop(0.0025)

#from weightnorm import AdamWithWeightnorm, data_based_init
#optimizer = AdamWithWeightnorm(0.001)
#data_based_init(model, X_train[:100])
#print('test')
model.compile(optimizer, 'categorical_crossentropy', sample_weight_mode="temporal")
model.save_weights('init_dense.hdf5')



#model = make_parallel(model, 2)
#model.compile(optimizer, 'categorical_crossentropy', sample_weight_mode="temporal")

history=History()


from keras.callbacks import LearningRateScheduler
import keras.backend as K

def scheduler(epoch):
    if epoch<=20:
        K.set_value(model.optimizer.lr, 0.0025)
    elif epoch<=21:
        K.set_value(model.optimizer.lr, 0.0001)
    return float(K.get_value(model.optimizer.lr))

lr_scheduler = LearningRateScheduler(scheduler)

from keras.callbacks import ReduceLROnPlateau
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=5, min_lr=0.000001, verbose=1)

checkpointer = ModelCheckpoint('weit/wave-adam-{epoch:02d}-{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1,
                                save_best_only=True, save_weights_only=True)

batch_size = 80#*2#16*2
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=100, verbose=1, sample_weight=train_mask,
          callbacks=[checkpointer,
                     lr_scheduler,
                     history],
          validation_data=(X_val, y_val, val_mask))


Using TensorFlow backend.


Train on 5600 samples, validate on 256 samples
Epoch 1/100
5600/5600 [==============================] - 64s - loss: 1.5780 - val_loss: 1.3399
Epoch 2/100
5600/5600 [==============================] - 53s - loss: 1.1692 - val_loss: 1.0071
Epoch 3/100
5600/5600 [==============================] - 54s - loss: 1.0560 - val_loss: 0.9668
Epoch 4/100
5600/5600 [==============================] - 53s - loss: 1.0029 - val_loss: 1.0324
Epoch 5/100
5600/5600 [==============================] - 53s - loss: 0.9699 - val_loss: 0.9387
Epoch 6/100
5600/5600 [==============================] - 53s - loss: 0.9529 - val_loss: 0.9492
Epoch 7/100
5600/5600 [==============================] - 53s - loss: 0.9227 - val_loss: 0.8789
Epoch 8/100
5600/5600 [==============================] - 53s - loss: 0.9082 - val_loss: 0.9162
Epoch 9/100
5600/5600 [==============================] - 53s - loss: 0.8987 - val_loss: 0.8964
Epoch 10/100
5600/5600 [==============================] - 53s - loss: 0.8760 - val_loss: 0.9129
Ep

In [ ]:
pickle.dump(history.history, open('wev-100-12-history-schedule.p','wb'))
#history2=pickle.load(open('wavenet8-history-schedule.p','rb'))

model.save_weights('wev-100-12-last-epoch-100.hdf5')
#model.load_weights('wavenet8-rmsprop-sched-93-0.7825.hdf5')

#
history2=history.history
import matplotlib.pyplot as plt
#get_ipython().magic(u'matplotlib inline')
plt.plot(xrange(len(history2['loss'])), history2['loss'])
plt.plot(xrange(len(history2['val_loss'])), history2['val_loss'])


#
cb_=np.load('cb513+profile_split1.npy')
cb = np.reshape(cb_,(514,700,57))
X_cb =cb[:,:, features]
y_cb =cb[:,:, labels]
cb_mask = cb[:,:,30] * -1 + 1


ypcb=model.predict(X_cb, 64)
masked_cb_accuracy(y_cb, ypcb)